In [ ]:
# pip install azureml-core
# pip install azureml azureml-sdk 
# pip install azureml-mlflow

In [1]:
from azureml.core import Workspace

c:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# create a new workspace or access existing workspace
try:
    ws = Workspace.create(name="aml-anshu",
                            subscription_id="84a5808b-5549-459a-98f2-f102e84fa1bb",
                            resource_group="EY23",
                            create_resource_group=True,
                            location='westus')

except:
    # access existing workspace
    ws = Workspace.get(name="aml-anshu",
                        subscription_id="84a5808b-5549-459a-98f2-f102e84fa1bb",
                        resource_group="EY23")
finally:
    print(ws)

Workspace.create(name='aml-anshu', subscription_id='84a5808b-5549-459a-98f2-f102e84fa1bb', resource_group='EY23')


In [3]:
ws

Workspace.create(name='aml-anshu', subscription_id='84a5808b-5549-459a-98f2-f102e84fa1bb', resource_group='EY23')

In [4]:
import pandas  as pd
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 
import mlflow 
import mlflow.sklearn
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [5]:
# data processing
df = pd.read_csv(r"Bank_churn_modelling.csv")
x = df[['CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'IsActiveMember']]
y =df['Exited']
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,stratify=y,random_state=5)

transformer = ColumnTransformer([('ohe',OneHotEncoder(drop="first"),[1,2]),],remainder='passthrough')
#transformer.fit(xtrain)

In [6]:
ws.get_mlflow_tracking_uri()

'azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EY23/providers/Microsoft.MachineLearningServices/workspaces/aml-anshu?'

In [7]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [8]:
mlflow.set_experiment("bank_churn")

2023/05/10 11:49:08 INFO mlflow.tracking.fluent: Experiment with name 'bank_churn' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1683699549158, experiment_id='d6c1be39-8a06-4aad-98bd-414219c3e861', last_update_time=None, lifecycle_stage='active', name='bank_churn', tags={}>

In [9]:
evaldata = xtest
evaldata['label'] = ytest

In [11]:

mlflow.sklearn.autolog()
with mlflow.start_run():
    model_pipeline = Pipeline([("transformer",transformer),
                                ("model",DecisionTreeClassifier(criterion='gini',min_samples_leaf=20,max_depth=8,
                               class_weight='balanced',random_state=5))])
    model_pipeline.fit(xtrain,ytrain)

    model_uri = mlflow.get_artifact_uri("model")
    #model evaluation

    result = mlflow.evaluate(model=model_uri,data=evaldata,
    targets='label',model_type='classifier',
    evaluators=['default'],
    evaluator_config={"default":{"metric_prefix":"test_"}}
    )

2023/05/10 11:51:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\anshu\AppData\Roaming\Python\Python310\site-packages\mlflow\models\signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/05/10 11:51:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\ProgramData\anaconda3\lib\si

<Figure size 1050x700 with 0 Axes>

In [12]:
from mlflow.models.signature import infer_signature,ModelSignature
from mlflow.types.schema import Schema, ColSpec

In [13]:

mlflow.sklearn.autolog()
with mlflow.start_run():
    model_pipeline = Pipeline([("transformer",transformer),
                                ("model",DecisionTreeClassifier(criterion='gini',min_samples_leaf=20,max_depth=8,
                               class_weight='balanced',random_state=5))])
    model_pipeline.fit(xtrain,ytrain)

    model_uri = mlflow.get_artifact_uri("model")
    #model evaluation

    result = mlflow.evaluate(model=model_uri,data=evaldata,
    targets='label',model_type='classifier',
    evaluators=['default'],
    evaluator_config={"default":{"metric_prefix":"test_"}},
    )


    # specify signature manually
    input_schema = Schema([
        ColSpec("double",'CreditScore'),
        ColSpec("string",'Geography'),
        ColSpec("string",'Gender'),
        ColSpec("double",'Age'),
        ColSpec("double",'Balance'),
        ColSpec("double",'NumOfProducts'),
        ColSpec("integer",'IsActiveMember'),
    ])
    output_schema = Schema([ColSpec("integer",'Exited'),])
    signature2 = ModelSignature(inputs=input_schema,outputs=output_schema)

    example = {"CreditScore":652.0,"Geography":'France',"Gender":"Male","Age":45.0,
                "Balance":152432.0,"NumOfProducts":2.0,"IsActiveMember":0}
    mlflow.sklearn.log_model(model_pipeline,"churn_model",
                                signature = signature2,
                                input_example=example
                                )

2023/05/10 12:18:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\anshu\AppData\Roaming\Python\Python310\site-packages\mlflow\models\signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/05/10 12:18:34 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/05/10 12:18:34 INFO mlfl

In [14]:
last_run = mlflow.last_active_run()
runid = last_run.info.run_id
runid

'1352a106-f9f0-42d5-a13e-2749d5339050'

In [15]:
# model registration
model_uri = f"runs:/{runid}/churn_model"
model_reg_name = "churn_prediction_name"
mlflow.register_model(model_uri,model_reg_name)

Successfully registered model 'churn_prediction_name'.
2023/05/10 12:19:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: churn_prediction_name, version 1
Created version '1' of model 'churn_prediction_name'.


<ModelVersion: aliases=[], creation_timestamp=1683701366925, current_stage='None', description='', last_updated_timestamp=1683701366925, name='churn_prediction_name', run_id='1352a106-f9f0-42d5-a13e-2749d5339050', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EY23/providers/Microsoft.MachineLearningServices/workspaces/aml-anshu/experiments/d6c1be39-8a06-4aad-98bd-414219c3e861/runs/1352a106-f9f0-42d5-a13e-2749d5339050/artifacts/churn_model', status='READY', status_message='', tags={}, user_id='', version='1'>

## Deployment over ACI

In [16]:
import json
from mlflow.deployments import get_deploy_client
deploy_config_path = "aciconfig.json"

client = get_deploy_client(ws.get_mlflow_tracking_uri())


In [17]:
# deployment configuration
config = {"deploy-config-file":deploy_config_path}
model_name = "churn_prediction_name"
model_version=1

In [18]:
webservice = client.create_deployment(model_uri = f"models:/{model_name}/{model_version}",
                            config=config,
                            name="churn-aci")


Running
2023-05-10 12:38:47+05:30 Creating Container Registry if not exists......

In [ ]:
webservice.scoring_uri